## Exploring Gradient Descent, Softmax Regression and Regularization Techniques using IRIS Data Set

IRIS is a simple dataset that comprises the `sepal` and `petal` dimensions of 3 types of IRIS flowers. 

I'll be using this dataset to test out a couple of algorithms for **classification** i.e. `Logistic Regression` and `Softmax Regression`.

I'll be exploring 3 versions of **Gradient Descent** i.e. `Batch`, `Stochastic` and `Mini Batch`

I'll be also be experiementing with a few techniques for **regularization** i.e. `Ridge`, `Lasso`, `Elastic Net`. I'll also be creating a version of the regularization that stops the training process as soon as the generalization error starts increasing i.e. `Early Stopping` 

<br>

Given the large set of things on agenda, the table of contents below should be helpful.



**Table of Contents**
1. Exploring IRIS Data
2. Preparing the Data for Logistic *(k=2)* and Multiclass *(k=3)* usecases


In [ ]:
<apple mango>